<a href="https://colab.research.google.com/github/adarsh-meher/MachineHack---Predict-News-Category/blob/master/MH_Predict_News_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fasttext
!pip install torchtext
!pip install transformers

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2385709 sha256=e486afe0f870545a7ab72d2ec44341cba00b70097e544ec46d1420c0ecfff327
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext
     |████████████████████████████████| 573kB 4.8MB/s 
     |████████████████████████████████| 890kB 22.7MB/s 
     |████████████████████████████████| 3.7MB 18.6MB/s 
     |████████████████████████████████| 1.0MB 49.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=093a75c5694b67298b04d9d01d9f12fd7edadae53eaba1ed2b850d00197910cf
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [3]:
!git clone https://github.com/adarsh-meher/MachineHack---Predict-News-Category.git
import os
os.chdir('/content/MachineHack---Predict-News-Category/')

Cloning into 'MachineHack---Predict-News-Category'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [4]:
import torch
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sb
import matplotlib.pyplot as plt
import time
import nltk
import gensim as G
import spacy as S
import fasttext as ft

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
news_data = pd.ExcelFile('Data_Train.xlsx').parse('Sheet1')

In [7]:
news_data.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [8]:
#### Counts of news category
print(news_data['SECTION'].value_counts()/news_data.shape[0])

'''
Politics: 0
Technology: 1
Entertainment: 2
Business: 3
'''

### no imbalance in categories

target_to_name_map = {0:'Politics',1:"Technology",2:"Entertainment",3:"Business"}

1    0.363398
2    0.252229
0    0.221028
3    0.163346
Name: SECTION, dtype: float64


In [0]:
#### Creating a test set assuming similar proportion of categories in target
news_train,news_test,target_train,target_test = train_test_split(news_data.drop(['SECTION'],axis = 1),news_data['SECTION'],test_size = 0.2,stratify = news_data['SECTION'])
news_train.index,target_train.index = range(news_train.shape[0]),range(news_train.shape[0])
news_test.index,target_test.index = range(news_test.shape[0]),range(news_test.shape[0])

## ***Analyzing news_train texts***

In [0]:
df = pd.concat([news_train.rename(columns = {'STORY' : 'text'}),pd.Series(target_train,name = 'target')],axis = 1)

In [11]:
df.head()

,text,target
0,The 10-year bond yield was trading at 7.4% fro...,3
1,"Further, Google said it verifies the identity ...",0
2,New Delhi: Social media intermediaries and the...,0
3,The national party which is gradually gaining ...,0
4,Modi will hold three public meetings in the po...,0


In [12]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()
ls = LancasterStemmer()
wns = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
##### Freq Dist of all words for all categories except stopwords
tokens = [ y for x in df['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha()]
tokens_fdist = FreqDist(tokens)

In [14]:
#### Freq distribution of words for each category
token_cat = {}
token_cat_fdist = {}
for i in df['target'].unique():
  print(target_to_name_map[i])
  token_cat[target_to_name_map[i]] = [ y for x in df[df['target'] == i]['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha() ]
  token_cat_fdist[target_to_name_map[i]] = FreqDist(token_cat[target_to_name_map[i]]) 

Business
Politics
Technology
Entertainment


In [0]:
#### Dataframe to show for each category the top n words
n = 100
top_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common(n)],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common(n)],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common(n)],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common(n)]},index=range(n))

In [16]:
top_words_each_cat

,Technology,Politics,Entertainment,Business
0,also,BJP,film,said
1,said,Congress,said,year
2,India,said,also,India
3,new,party,like,market
4,Apple,Modi,actor,growth
...,...,...,...,...
95,sale,come,things,order
96,products,ruling,among,cut
97,variant,issue,still,increase
98,system,crore,America,rates


In [0]:
#### Extract set of words that appear both of any of the two categories
from itertools import combinations
common_words = []
for i in combinations(target_to_name_map.values(),2):
  [ common_words.append(x) for x in set(top_words_each_cat[i[0]]).intersection(set(top_words_each_cat[i[1]])) ]

common_words = set(list(common_words))

In [0]:
#### Dataframe to show for each category the bottom n words
n = 100
least_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common()[-n:]],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common()[-n:]],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common()[-n:]],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common()[-n:]]},index=range(n))

In [19]:
least_words_each_cat

,Technology,Politics,Entertainment,Business
0,furore,equidistance,asusual,James
1,pornographic,KV,Entire,Petersburg
2,performers,axing,GREAT,ought
3,justified,Hibi,Chhabra,tell
4,Shradha,Eden,Jus,slamming
...,...,...,...,...
95,Quora,leher,benefiting,accompanied
96,similarity,Mehiwal,grade,reminiscent
97,nationally,Chandan,setup,fakes
98,bound,Gappu,kickstarted,Aniruddha


# **Cleaning text data**

In [0]:
from html.parser import HTMLParser
import re
import string

In [0]:
contrac = {'s' : 'is','t':'not','ve' : 'have','d' : 'had', '':'a'}

def clean_text(x):
  ###### Remove HTML characters
  x = HTMLParser().unescape(x)

  ##### Treating apostrophes/contraction words
  #x = ' '.join([ w.split('\'')[0].lower()[:-1]+' '+contrac[w.split('\'')[1].lower()] if '\'' in w else w for sent in sent_tokenize(x) for w in sent.split()  ])
  word_list = []
  for sent in sent_tokenize(x):
    for w in sent.split():
      if '\'' in w:
        w1 = w.split('\'')[0].lower()[:-1]
        try:
          c1 = w.split('\'')[1]/lower()
          w_new = w1 + ' '+ contrac[w2]
          word_list.append(w_new)
        except:
          w_new = w1
          word_list.append(w_new)
      else:
        word_list.append(w)
    
  x = ' '.join(word_list) 

  ##### Removing stopwords and stemming words to its base form
  x = ' '.join([ ps.stem(w) for w in word_tokenize(x) if w not in stop_words])

  ##### Remove punctuations
  table = str.maketrans('', '', string.punctuation)
  x = ' '.join([w.translate(table) for w in word_tokenize(x) ])

  ##### Replace digits with 'numeral' string to denote presence of number. This can be used in business category as presence of number in that category is certainly possible
  x = ' '.join([ 'numeral' if w.isdigit() else w for w in word_tokenize(x)])

  ##### Remove presence of url/https links
  url_pattern = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
  x = re.sub(url_pattern, ' ', x)

  ##### Remove nonascii characters
  x = ''.join([chr for chr in x if ord(chr) < 128])

  ##### strip and leading/training spaces and convert all words to lowercase
  x = x.strip().lower()

  return x

  

In [22]:
news_train['clean_text'] = news_train['STORY'].apply(lambda x : clean_text(x))
news_test['clean_text'] = news_test['STORY'].apply(lambda x : clean_text(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


In [0]:
pd.concat([target_train,news_train.drop(['STORY'],axis = 1)],axis = 1).to_csv('News_Train.csv',index = False)
pd.concat([target_test,news_test.drop(['STORY'],axis = 1)],axis = 1).to_csv('News_Test.csv',index = False)

### **`Classification using Torch`**

Simple LSTM Baseline model using TorchText

In [0]:
import torch 
from torchtext.data import Field,BucketIterator,TabularDataset,LabelField

In [0]:
text_field = Field(tokenize = word_tokenize,lower = True)
label_field = LabelField(use_vocab = False)
datafields = [('SECTION',label_field),('clean_text',text_field)]

In [0]:
train = TabularDataset(path = os.path.join(os.getcwd(),'News_Train.csv'),format = "csv",skip_header=False,fields=datafields)
test = TabularDataset(path = os.path.join(os.getcwd(),'News_Test.csv'),format = "csv",skip_header=False,fields=datafields)

In [0]:
text_field.build_vocab(train,max_size = 5000)

In [0]:
train_iter,test_iter = BucketIterator.splits((train,test),batch_size=(64,64),sort_key=lambda x: len(x.clean_text),sort_within_batch=False)